# 01. Data Cleaning and Feature Engineering


---

## 1. Introduction

The goal of this notebook is to clean the consolidated dataset created in the previous step and to engineer new features that will be useful for the exploratory data analysis (EDA).

The workflow is as follows:
1.  Load the processed `main_data.parquet` file.
2.  Inspect the data for inconsistencies, missing values, and incorrect data types.
3.  Clean the data by handling missing values and correcting data types.
4.  Engineer new features, such as calculating shipping times and extracting temporal information (month, day of the week).
5.  Save the final, clean, and enriched dataframe to the `processed` folder.

In [1]:
# --- 2. Setup and Imports ---

# Importing standard libraries
import pandas as pd
import numpy as np
import os
import sys

# Adding the project's root directory to the Python path
sys.path.append('..')

# Importing our custom functions
from src.data_utils import load_processed, save_processed
from src.features import compute_shipping_time, add_order_value # Importing our new feature functions

# Configuring pandas for better display
pd.set_option('display.max_columns', 100)

print("Setup complete.")

Setup complete.


In [2]:
# --- 3. Loading Processed Data ---

# Load the main consolidated dataframe from the processed folder
df = load_processed('main_data')

# --- Verification ---
print("--- Initial Data Verification ---")
print(f"DataFrame loaded successfully. Shape: {df.shape}")
print("\n[INFO]")
df.info()
print("\n[HEAD]")

display(df.head())

Loading processed data from: /home/lucas/olist-data-analysis-project/notebooks/../data/processed/main_data.parquet
--- Initial Data Verification ---
DataFrame loaded successfully. Shape: (119143, 40)

[INFO]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   customer_id                    119143 non-null  object 
 2   order_status                   119143 non-null  object 
 3   order_purchase_timestamp       119143 non-null  object 
 4   order_approved_at              118966 non-null  object 
 5   order_delivered_carrier_date   117057 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  119143 non-null  object 
 8   order_item_id                  118310 non-null  float64
 9   produ

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,None,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,2.00,a54f0611adc9ed256b57ede6b6eb5114,4.0,None,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,18.59,a54f0611adc9ed256b57ede6b6eb5114,4.0,None,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,perfumery
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,1.0,credit_card,3.0,179.12,e73b67b67587f7644d5bd1a52deb1b01,5.0,None,None,2018-08-18 00:00:00,2018-08-22 19:07:58,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,auto


---
## 4. Data Cleaning

Here, we will perform the initial data cleaning by calling our dedicated cleaning function from `src/cleaning.py`. This function handles operations like correcting data types across the entire dataframe.

In [3]:
# Import the new cleaning function
from src.cleaning import clean_data

# Apply the cleaning function to our dataframe
df_clean = clean_data(df)

# --- Verification ---
print("\n--- Verification after cleaning ---")
# Check the data types of a few key columns to confirm the change
datetime_cols_to_check = [
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
df_clean[datetime_cols_to_check].info()

--- Data Cleaning Step ---
Converting timestamp columns to datetime objects...
  - Column 'order_purchase_timestamp' converted.
  - Column 'order_approved_at' converted.
  - Column 'order_delivered_carrier_date' converted.
  - Column 'order_delivered_customer_date' converted.
  - Column 'order_estimated_delivery_date' converted.
  - Column 'shipping_limit_date' converted.
  - Column 'review_creation_date' converted.
  - Column 'review_answer_timestamp' converted.
Data cleaning complete.

--- Verification after cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 3 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_purchase_timestamp       119143 non-null  datetime64[ns]
 1   order_delivered_customer_date  115722 non-null  datetime64[ns]
 2   order_estimated_delivery_date  119143 non-null  datetime64[ns]
dtypes: datetime64[ns

### 4.2. Handling Missing Values

Now that the data types are correct, we'll investigate the presence of missing values (`NaN` or `NaT`) in our dataset. Our goal is to understand which columns have missing data, how much is missing, and then decide on a strategy for each case (e.g., removing the rows, filling with a specific value, or leaving them as is).

In [4]:
# Calculate the total count and percentage of missing values for each column
missing_values = df_clean.isnull().sum()
missing_percentage = (missing_values / len(df_clean)) * 100

# Create a summary dataframe for missing values
missing_summary = pd.DataFrame({
    'missing_count': missing_values,
    'missing_percentage': missing_percentage
})

# Filter to show only columns with missing values and sort by percentage
missing_summary_filtered = missing_summary[missing_summary['missing_count'] > 0].sort_values(
    by='missing_percentage',
    ascending=False
)

# --- Verification ---
print("--- Missing Values Analysis ---")
print("Columns with missing data, sorted by percentage:")
display(missing_summary_filtered)

--- Missing Values Analysis ---
Columns with missing data, sorted by percentage:


,missing_count,missing_percentage
review_comment_title,105154,88.258647
review_comment_message,68898,57.827988
order_delivered_customer_date,3421,2.871339
product_category_name_english,2567,2.154554
product_name_lenght,2542,2.133571
product_photos_qty,2542,2.133571
product_description_lenght,2542,2.133571
product_category_name,2542,2.133571
order_delivered_carrier_date,2086,1.750837
review_answer_timestamp,997,0.836810


### 4.3. Missing Values Treatment Strategy

Based on the analysis above, the following strategy will be implemented:

1.  **Critical Key Columns (`product_id`, `seller_id`):** Rows with missing values in these columns will be **removed**. These records are incomplete and cannot be reliably used in product- or seller-level analysis. The data loss is minimal (~0.7%).
2.  **Product Category Columns:** Missing values in `product_category_name` and `product_category_name_english` will be **filled** with the placeholder string `"uncategorized"`. This preserves the sales data associated with these products.
3.  **Review Comments & Delivery Dates:** Missing values in `review_comment_title`, `review_comment_message`, and `order_delivered_customer_date` will be **kept as is**. They represent valid states (no comment was left, or the order is still in transit) and are important for specific analyses.

In [5]:
# --- Handling Missing Key Columns ---

print(f"Shape before dropping NA in key columns: {df_clean.shape}")

# Drop rows where 'product_id' or 'seller_id' are null
# These are essential for linking products and sales
df_clean.dropna(subset=['product_id', 'seller_id'], inplace=True)

print(f"Shape after dropping NA: {df_clean.shape}")

# --- Verification ---
print("\nVerifying that missing values in key columns are removed:")
print(f"Missing values in 'product_id' and 'seller_id': {df_clean['product_id'].isnull().sum()}, {df_clean['seller_id'].isnull().sum()}")


Shape before dropping NA in key columns: (119143, 40)


Shape after dropping NA: (118310, 40)

Verifying that missing values in key columns are removed:
Missing values in 'product_id' and 'seller_id': 0, 0


In [6]:
# --- Handling Missing Product Categories ---

# Define the columns to fill
category_cols_to_fill = [
    'product_category_name',
    'product_category_name_english',
    # We can also fill the product dimension columns with 0 or the median,
    # but for now, we'll focus on the categorical ones.
]

# Define the placeholder value
placeholder = 'uncategorized'

print(f"\nFilling NA in category columns with '{placeholder}'...")

for col in category_cols_to_fill:
    df_clean[col].fillna(placeholder, inplace=True)
    print(f"  - Missing values filled for '{col}'.")

# --- Verification ---
print("\nVerifying that missing values in category columns are filled:")
print(f"Missing values in 'product_category_name'and 'product_category_name_english': {df_clean['product_category_name'].isnull().sum()}, {df_clean['product_category_name_english'].isnull().sum()}")



Filling NA in category columns with 'uncategorized'...
  - Missing values filled for 'product_category_name'.
  - Missing values filled for 'product_category_name_english'.

Verifying that missing values in category columns are filled:
Missing values in 'product_category_name'and 'product_category_name_english': 0, 0


/tmp/ipykernel_756347/1053434633.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].fillna(placeholder, inplace=True)


---
## 5. Feature Engineering

With a clean dataset, we can now create new features to enrich our analysis. These features will help us calculate durations, delays, and other valuable metrics that are not present in the original data. We will use the custom functions defined in `src/features.py`.

In [7]:
# --- 5.1. Creating New Features ---

# Re-importing our feature functions in case they were updated
from src.features import (
    add_order_value,
    compute_shipping_time,
    compute_shipping_delay,
    compute_delivery_total_time
)

# Create a copy to work on, which is a good practice to avoid chained assignment warnings
df_featured = df_clean.copy()

print("Starting feature engineering...")

# Apply each feature engineering function
df_featured = add_order_value(df_featured)
print("  - 'order_value' created.")

df_featured = compute_shipping_time(df_featured)
print("  - 'shipping_time_days' created.")

df_featured = compute_delivery_total_time(df_featured)
print("  - 'total_delivery_time' created.")

df_featured = compute_shipping_delay(df_featured)
print("  - 'shipping_delay_days' created.")

# --- Verification ---
print("\n--- Verification of New Features ---")
# List of newly created columns to inspect
new_feature_columns = [
    'order_value',
    'shipping_time_days',
    'shipping_delay_days',
    'total_delivery_time'
]

# Display the head of the new columns to verify their creation and values
display(df_featured[['order_id'] + new_feature_columns].head())



Starting feature engineering...
  - 'order_value' created.
  - 'shipping_time_days' created.
  - 'total_delivery_time' created.
  - 'shipping_delay_days' created.

--- Verification of New Features ---


,order_id,order_value,shipping_time_days,shipping_delay_days,total_delivery_time
0,e481f51cbdc54678b7cc49136f2d6af7,89.97,8,0,8
1,e481f51cbdc54678b7cc49136f2d6af7,89.97,8,0,8
2,e481f51cbdc54678b7cc49136f2d6af7,89.97,8,0,8
3,53cdb2fc8bc7dce0b6741e2150273451,118.70,13,0,13
4,47770eb9100c2d0c44946d9cf07ec65d,159.90,9,0,9


---
## 6. Saving the Enriched Data

The final step in this notebook is to save our cleaned and feature-enriched dataframe. This new dataset will be the foundation for all subsequent analysis and visualization notebooks. We'll save it with a new name to distinguish it from the merely merged data.

In [8]:
# --- 6.1. Saving the Final DataFrame for Analysis ---

# Define a name for our final, analytics-ready dataset
output_filename = 'analytics_main_data'

# Save the dataframe using our custom function
save_processed(df_featured, output_filename)


# --- Verification ---
print("\n--- Final Data Saving Verification ---")
final_file_path = f'../data/processed/{output_filename}.parquet'

if os.path.exists(final_file_path):
    # Load the saved file back to double-check its integrity
    df_check = pd.read_parquet(final_file_path)
    print(f"✅ Success: File '{output_filename}.parquet' saved correctly.")
    print(f"   - Shape of saved file: {df_check.shape}")
    print(f"   - Columns: {df_check.columns.tolist()}")
else:
    print(f"❌ Error: File not found at '{final_file_path}'.")

Data saved to: /home/lucas/olist-data-analysis-project/notebooks/../data/processed/analytics_main_data.parquet

--- Final Data Saving Verification ---
✅ Success: File 'analytics_main_data.parquet' saved correctly.
   - Shape of saved file: (118310, 44)
   - Columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value', 'review_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_heigh